# Capstone Project 1 - Connecting to Data

In [4]:
!pip install mysql.connector
import mysql.connector as mysql #if it gives error, use: pip install mysql.connector
import csv
import pandas as pd
import numpy as np

     |████████████████████████████████| 11.9MB 12.0MB/s 
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl size=247949 sha256=cfe33c20a938d326749c6af095bf1c26770e80f06ab0150cf05398220202fd7f
  Stored in directory: /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175
Successfully built mysql.connector


In [0]:
#Creating the connection string to connect to INSAID's mysql db

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

In [6]:
print(dbconnectionstring)

In [7]:
# List all tables in the database

cursor.execute("SHOW TABLES")
tables=cursor.fetchall()
for table_name in tables:
    print(table_name)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [8]:
# Table 1 - events_data - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from events_data"
cursor.execute(query)
result=cursor.fetchall()
result

[('event_id', 'bigint(20)', 'NO', '', None, ''),
 ('device_id', 'bigint(255)', 'YES', '', None, ''),
 ('timestamp', 'datetime', 'NO', '', None, ''),
 ('longitude', 'float(8,6)', 'YES', '', None, ''),
 ('latitude', 'float(8,6)', 'YES', '', None, ''),
 ('city', 'varchar(50)', 'NO', '', None, ''),
 ('state', 'varchar(50)', 'NO', '', None, '')]

In [9]:
# Table 2 - gender_age_train - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from gender_age_train"
cursor.execute(query)
result=cursor.fetchall()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('gender', 'varchar(2555)', 'NO', '', None, ''),
 ('age', 'int(255)', 'NO', '', None, ''),
 ('group', 'varchar(2555)', 'NO', '', None, '')]

In [10]:
# Table 3 - phone_brand_device_model - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from phone_brand_device_model"
cursor.execute(query)
result=cursor.fetchall()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('phone_brand', 'varchar(2555)', 'NO', '', None, ''),
 ('device_model', 'varchar(2555)', 'NO', '', None, '')]

In [11]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from phone_brand_device_model"
cursor.execute(query)
pcount = cursor.fetchall()
pcount


[(87726,)]

In [12]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from gender_age_train"
cursor.execute(query)
pcount = cursor.fetchall()
pcount


[(74645,)]

Query

In [0]:
# Get all records for the concerned States from events_data

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = '''select *
             from events_data as ed
            where state in ('MadhyaPradesh', 'Goa', 'Chhattisgarh','Uttaranchal', 'JammuandKashmir', 'Nagaland')'''
cursor.execute(query)
events_data=cursor.fetchall()
ds_events_data= pd.DataFrame(events_data)
ds_events_data.columns=[i[0] for i in cursor.description]

In [27]:
ds_events_data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
0,10117,-5.323073e+18,2016-05-01 11:11:24,75.939545,22.769171,Indore,MadhyaPradesh
1,112369,-5.323073e+18,2016-05-04 22:47:19,75.939545,22.769171,Indore,MadhyaPradesh
2,139098,-5.323073e+18,2016-05-01 09:55:58,75.939545,22.769171,Indore,MadhyaPradesh
3,190983,-5.323073e+18,2016-05-04 22:51:03,75.939545,22.769171,Indore,MadhyaPradesh
4,249507,-5.323073e+18,2016-05-01 11:24:12,75.939545,22.769171,Indore,MadhyaPradesh


In [28]:
# Create a unique list of device ids from event data, to download only those records from the other 2 tables.

device_ids = ds_events_data['device_id'].unique()    # Extracts unique device ids
dids=device_ids.tolist()                           # Converts nparray to list
list_of_ids = [x for x in dids if str(x) != 'nan'] # Drop all 'nan' ids
len(list_of_ids)                                   # list the number of ids.

4297

In [0]:
# Download the records from phone_brand_device_model

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(                     # Connection string used again, since it keeps disconnecting, so 
                      host = 'cpanel.insaid.co',        # executing this again, before every select statements.
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

format_strings = ','.join(['%s'] * len(list_of_ids))   # This needs to be added to pass the ids. if we have 5 ids, then it 
                                                       # will create 5, %s, positional parameters.

query = '''select * 
             from phone_brand_device_model
            where device_id in ( %s )''' % format_strings

cursor.execute(query, tuple(list_of_ids))
phone_data=cursor.fetchall()
ds_phone_data = pd.DataFrame(phone_data)
ds_phone_data.columns=[i[0] for i in cursor.description]

In [40]:
ds_phone_data.head()

,device_id,phone_brand,device_model
0,2313145512701915151,vivo,Xplay
1,-1539979963116391306,Xiaomi,GN708W
2,-7731743086143077949,Xiaomi,红米
3,-4370696809239943432,Xiaomi,MI 2S
4,2959524852145775517,Samsung,A850


In [20]:
len(list_of_ids)

4297

In [0]:
# Download the records from gender_age_train

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

format_strings = ','.join(['%s'] * len(list_of_ids))

#list_ids = tuple(','.join(['%s'] * len(ldevice_ids)))

query = '''select * 
             from gender_age_train
            where device_id in ( %s )''' % format_strings

cursor.execute(query,tuple(list_of_ids))
user_data=cursor.fetchall()
ds_user_data = pd.DataFrame(user_data)
ds_user_data.columns=[i[0] for i in cursor.description]

In [24]:
ds_user_data.head()

,device_id,gender,age,group
0,2313145512701915151,M,34,M32-38
1,-1539979963116391306,M,31,M29-31
2,-7731743086143077949,M,35,M32-38
3,-4370696809239943432,M,36,M32-38
4,2959524852145775517,M,30,M29-31


In [0]:
ds_phone_data.replace('小米','Xiaomi',inplace=True)
ds_phone_data.replace('华为','Huawei',inplace=True)
ds_phone_data.replace('三星','Samsung',inplace=True)
ds_phone_data.replace('魅族','Meizu',inplace=True)
ds_phone_data.replace('乐视','LeEco',inplace=True)
ds_phone_data.replace('酷派','Coolpad',inplace=True)
ds_phone_data.replace('联想 ','Lenovo',inplace=True)
ds_phone_data.replace('联想','Lenovo',inplace=True)

In [0]:
# Write all records from the data, to excel files

ds_events_data.to_excel('events_data.xlsx')
ds_phone_data.to_excel('phone_data.xlsx')
ds_user_data.to_excel('user_data.xlsx')

In [0]:
import pandas_profiling

In [31]:
ds_events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261097 entries, 0 to 261096
Data columns (total 7 columns):
event_id     261097 non-null int64
device_id    261046 non-null float64
timestamp    261097 non-null datetime64[ns]
longitude    261034 non-null float64
latitude     261034 non-null float64
city         261097 non-null object
state        261097 non-null object
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 13.9+ MB


In [32]:
ds_phone_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4297 entries, 0 to 4296
Data columns (total 3 columns):
device_id       4297 non-null int64
phone_brand     4297 non-null object
device_model    4297 non-null object
dtypes: int64(1), object(2)
memory usage: 100.8+ KB


In [33]:
ds_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4297 entries, 0 to 4296
Data columns (total 4 columns):
device_id    4297 non-null int64
gender       4297 non-null object
age          4297 non-null int64
group        4297 non-null object
dtypes: int64(2), object(2)
memory usage: 134.4+ KB


**Good News is all out three datasets dont have any null values.**


**Now we will do some panda pre-profiling on our datasets**

In [35]:
profile=pandas_profiling.ProfileReport(ds_events_data)
profile.to_file(outputfile="Events_data_before_preprocessing.html")

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Below are the findings from event data after running profiling.


1.   Maximum events were generated in Indore city
2.   Maximum events, state-wise were generated in Madhya Pradesh
3. Our dataset covers 85 distinct cities.



In [45]:
profile=pandas_profiling.ProfileReport(ds_phone_data)
profile.to_file(outputfile="Phone_data_before_preprocessing.html")

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Below are the findings from phone data after running profiling.


1.   Some of the phone brands still have name in foreign language.
2.   Some of the device model are also in foreign language.
3. There are 60 total phone brands with 676 device models

In [46]:
profile=pandas_profiling.ProfileReport(ds_user_data)
profile.to_file(outputfile="User_data_before_preprocessing.html")

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Below are the findings from phone data after running profiling.


1.   There are 2702 Males and 1595 females.
2.   The age distribution is slightly left skewed with our youngest cunstomer being 6 years old and oldest bein 90 years old.


In [0]:
from google.colab import drive
drive.mount('/content/drive')